In [86]:
!gdown 1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF

Downloading...
From: https://drive.google.com/uc?id=1-K5XbNHNLYBQG9VyTQUzxov_x5AmoITF
To: /content/IMDB_5.csv
100% 323k/323k [00:00<00:00, 24.1MB/s]


In [87]:
import pandas as pd

df = pd.read_csv('/content/IMDB_5.csv')
df

,review_es,sentiment
0,"¡Esta película es horrible, ni siquiera puedo ...",negative
1,Why do movie makers always go against the auth...,negative
2,No puedo creer que aquellos alabando esta pelí...,negative
3,Esta película realmente usó sus ubicaciones bi...,positive
4,"De manera extraña, esta película nunca llegó a...",positive
...,...,...
245,Obabakoak es un montón de cuentos con un único...,negative
246,Todo el mundo civilizado por ahora sabe que es...,positive
247,"Soy un enorme fan de las películas de guerra, ...",negative
248,"De acuerdo, aquí está el trato: si te ofenden ...",positive


In [88]:
!pip install transformers[torch] datasets evaluate

In [89]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/IMDB_5.csv")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review_es', 'sentiment'],
        num_rows: 250
    })
})

In [90]:
from sklearn.preprocessing import LabelEncoder

In [91]:
encoder = LabelEncoder()

In [92]:
dataset['train']['sentiment']

['negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',

In [93]:
dataset['train'] = dataset['train'].add_column('label', encoder.fit_transform(dataset['train']['sentiment']))

In [94]:
dataset['train'] = dataset['train'].remove_columns('sentiment')

In [95]:
dataset['train'].train_test_split(test_size=0.2)

DatasetDict({
    train: Dataset({
        features: ['review_es', 'label'],
        num_rows: 200
    })
    test: Dataset({
        features: ['review_es', 'label'],
        num_rows: 50
    })
})

In [96]:
dataset = dataset['train'].train_test_split(test_size=0.2)

In [97]:
def tokenize_function(examples):
    return tokenizer(examples["review_es"], padding="max_length", truncation=True)

In [98]:
from transformers import AutoTokenizer

In [99]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [100]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
import torch

In [102]:
from transformers import TrainingArguments

In [103]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    learning_rate=5e-5
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`